<a href="https://colab.research.google.com/github/VictorUceda/TrackNet-Padel/blob/main/cut_point.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
!git clone https://github.com/VictorUceda/TrackNet-Padel

remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (1/1), done.
remote: Total 3 (delta 2), reused 3 (delta 2), pack-reused 0
Unpacking objects: 100% (3/3), done.
From https://github.com/VictorUceda/TrackNet-Padel
   8321e08..0ebc4c4  main       -> origin/main
Updating 8321e08..0ebc4c4
Fast-forward
 cut_fragments.py | 2 +-
 1 file changed, 1 insertion(+), 1 deletion(-)


In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

video_path = '/content/gdrive/MyDrive/PadelMatchs/WPTDanish_Octavos_PD_0_01.mp4'
points_path = video_path[:-4]+'.points.csv'

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
%cd TrackNet-Padel

/content/TrackNet-Padel


In [4]:
import pandas as pd

df_segments = pd.read_csv(points_path, sep=';')
i = 0
for index, row in df_segments.iterrows():
  name = video_path[:-4]+'_'+str(i)+'.mp4'
  ini = (int(row['ini']/25/60), (row['ini']%(25*60))/25)
  dur = (int((row['fin']-row['ini'])/25/60), ((row['fin']-row['ini'])%(25*60))/25)
  print('00:'+str(ini[0])+':'+str(ini[1]), '00:'+str(dur[0])+':'+str(dur[1]), name)
  !ffmpeg -y -ss {'00:'+str(ini[0])+':'+str(ini[1])} -i {video_path} -c copy -t {'00:'+str(dur[0])+':'+str(dur[1])} {name}
  i += 1

00:0:0.0 00:0:10.0 /content/gdrive/MyDrive/PadelMatchs/WPTDanish_Octavos_PD_0_01_0.mp4
ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libw